In [1]:
import os
import missionbio.mosaic.io as mio
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from kneed import KneeLocator
import math
import ipywidgets as widgets
from IPython.display import display
import quilt3 as q3
import io

In [ ]:
#Suggested PCA and K-means clusters using elbow/knee plots.

def automated_mosaic_cnv(filepath_to_h5):

    sample = mio.load(filepath_to_h5)
    ids=sample.cnv.get_attribute('id')
    reads = sample.cnv.get_attribute('read_counts', constraint = 'row+col')
    working_amplicons = (reads.median() > 0).values
    sample.cnv = sample.cnv[:, working_amplicons]
    sample.cnv.normalize_reads()

    pca = PCA().fit(sample.cnv.get_attribute('normalized_counts'))
    
    plt.rcParams["figure.figsize"] = (14,9)

    fig, ax = plt.subplots()
    xi = np.arange(1, len(np.cumsum(pca.explained_variance_ratio_))+1, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)

    plt.plot(xi,y)
    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, len(y)+1, step=1)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.99, color='r', linestyle='-')
    plt.text(0.95, 0.95, '99% cut-off threshold', color = 'red', fontsize=16)
    
    ax.grid(axis='x')
    
    var_x=[]
    var_y=[]
    for x in range(len(xi)):
        var_x.append(0)
        var_y.append(0.99)

    idx = np.argwhere(np.diff(np.sign(y - var_y))).flatten()
    plt.plot(xi[idx], y[idx], 'ro')
    plt.show()

    num_of_components=math.ceil(int(xi[idx]))
    

    return sample,ids,num_of_components

    

def automated_mosaic_protein(filepath_to_h5):
    sample = mio.load(filepath_to_h5)
    ids = sample.protein.get_attribute('id')
    sample.protein.normalize_reads('CLR')

    pca = PCA().fit(sample.protein.get_attribute('normalized_counts'))

    plt.rcParams["figure.figsize"] = (14,9)

    fig, ax = plt.subplots()
    xi = np.arange(1, len(np.cumsum(pca.explained_variance_ratio_))+1, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)

    plt.plot(xi,y)
    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, len(y)+1, step=1))
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.99, color='r', linestyle='-')
    plt.text(0.95, 0.95, '99% cut-off threshold', color = 'red', fontsize=16)
    
    ax.grid(axis='x')
    
    var_x=[]
    var_y=[]
    for x in range(len(xi)):
        var_x.append(0)
        var_y.append(0.99)

    idx = np.argwhere(np.diff(np.sign(y - var_y))).flatten()
    plt.plot(xi[idx], y[idx], 'ro')
    plt.show()
    
    num_of_components=math.ceil(int(xi[idx]))
    
    return sample,ids,num_of_components



In [ ]:
#Analysis using parameters that the user selects
def cnv_analysis(filepath):
    print('\033[1m',"DNA/CNV Analysis:"+'\033[0m')
    sample_cnv,ids_cnv,suggested_PCA=automated_mosaic_cnv(filepath)
    
    print("Suggested Parameter From Variance Plot Above: ")
    print("PCA components: "+str(suggested_PCA))
    print("\n")

    listofid=[]
    for element in ids_cnv.values:
        stripped_id=str(element).replace("[","")
        stripped_id2=str(stripped_id).replace("]","")
        stripped_id3=str(stripped_id2).replace("'","")
        listofid.append(str(stripped_id3))

    print("Select The Number of PCA Components: ")
    pca_comps_cnv=widgets.BoundedIntText(min=2,max=100,step=1,description='PCA Components:',disabled=False)
    display(pca_comps_cnv)
    button = widgets.Button(description="Parameter Selected")
    display(button)
    print("\n")
    out = widgets.Output()
    display(out)

    @out.capture(clear_output=True,wait=True)
    def on_button_clicked(b):
        sample_cnv.cnv.run_pca(components = pca_comps_cnv.value, attribute = 'normalized_counts', show_plot = False)
        sample_cnv.cnv.run_umap(attribute = 'pca')
        
        print("\n")
        Sum_of_squared_distances = []
    
        K = range(1,40)
        for k in K:
            km = KMeans(n_clusters=k)
            km = km.fit(sample_cnv.cnv.get_attribute('umap'))
            Sum_of_squared_distances.append(km.inertia_)
    
        plt.plot(K, Sum_of_squared_distances, 'bx-')
        plt.xlabel('k')
        plt.ylabel('Sum of Squared Distances')
        plt.title('Elbow Method For Optimal k')
        kn = KneeLocator(K, Sum_of_squared_distances, curve='convex', direction='decreasing')
        plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
        plt.show()
        print("Suggested Number of K-Means Clusters for "+ str(pca_comps_cnv.value)+" PCA Components: "+str(kn.knee))
        print("\n")
        
    
    button.on_click(on_button_clicked)
    
    print("Select The Number of K-Means Clusters: ")
    kmeans_clusters_cnv=widgets.BoundedIntText(min=2,max=100,step=1,description='K-Means Clusters:',disabled=False)
    display(kmeans_clusters_cnv)
    button = widgets.Button(description="Parameter Selected")
    display(button)
    print("\n")
    out = widgets.Output()
    display(out)

    @out.capture(clear_output=True,wait=True)
    def on_button_clicked(b):
        
        sample_cnv.cnv.cluster(attribute = 'umap', method = 'kmeans',n_clusters = kmeans_clusters_cnv.value)
   
        fig1 = sample_cnv.cnv.scatterplot(attribute='umap', colorby='label') 
        fig1.show()
        print("\n")
    
    
        fig2 = sample_cnv.cnv.heatmap(attribute="read_counts") 
        fig2.show()
        print("\n")
    
    button.on_click(on_button_clicked)

    print("Select Features - Use the Command Button for Multiple Selections: ")
    features_list_cnv=widgets.SelectMultiple(options=listofid,description='Select Features',disabled=False)
    display(features_list_cnv)
    button = widgets.Button(description="Features Selected")
    display(button)
    print("\n")
    out = widgets.Output()
    display(out)

    @out.capture(clear_output=True,wait=True)
    def on_button_clicked(b):
        print(list(features_list_cnv.value))
        if list(features_list_cnv.value): 
        
            fig3 = sample_cnv.cnv.scatterplot(attribute='umap',colorby='read_counts', features=features_list_cnv.value) 
            fig3.show()
        
#             button = widgets.Button(description="Save plot")
#             display(button)
            print("\n")
#             out = widgets.Output()
#             display(out)

#             @out.capture(clear_output=True,wait=True)
#             def on_button_clicked(b):
#                 fig3.write_image("dna_feature_umap.png")
#                 print("Saved image as dna_feature_umap.png")
            
#             button.on_click(on_button_clicked)
    
            fig4 = sample_cnv.cnv.heatmap(attribute='read_counts',features=features_list_cnv.value) 
            fig4.show()
        
#             button = widgets.Button(description="Save plot")
#             display(button)
            print("\n")
#             out = widgets.Output()
#             display(out)

#             @out.capture(clear_output=True,wait=True)
#             def on_button_clicked(b):
#                 fig4.write_image("dna_feature_heatmap.png")
#                 print("Saved image as dna_feature_heatmap.png")
            
#             button.on_click(on_button_clicked)
    
    button.on_click(on_button_clicked)

In [ ]:
def protein_analysis(filepath):
    print('\n')
    print('\033[1m',"Protein Analysis: "+'\033[0m')
    print('*******************************************************************')
    print('PCA Components = 10 is appropriate for a 45-plex Biolegend panel.')
    print('Select fewer components when analyzing custom panels with 2-20 AOCs.')
    print('*******************************************************************')
    
    sample_protein,ids_protein,suggested_PCA_protein=automated_mosaic_protein(filepath)
    
    print("Suggested Parameter From Variance Plot Above: ")
    print("PCA components: "+str(suggested_PCA_protein))
    print("\n")

    listofid=[]
    for element in ids_protein.values:
        stripped_id=str(element).replace("[","")
        stripped_id2=str(stripped_id).replace("]","")
        stripped_id3=str(stripped_id2).replace("'","")
        listofid.append(str(stripped_id3))

    print("Select The Number of PCA Components: ")
    pca_comps_protein=widgets.BoundedIntText(min=2,max=100,step=1,description='PCA Components:',disabled=False)
    display(pca_comps_protein)
    button = widgets.Button(description="Parameter Selected")
    display(button)
    print("\n")
    out = widgets.Output()
    display(out)

    @out.capture(clear_output=True,wait=True)
    def on_button_clicked(b):
        sample_protein.protein.run_pca(attribute = 'normalized_counts', components = pca_comps_protein.value, show_plot = False)
        sample_protein.protein.run_umap(attribute = 'pca',random_state = 42)
        
        print("\n")
        
        Sum_of_squared_distances = []
    
        K = range(1,40)
        for k in K:
            km = KMeans(n_clusters=k)
            km = km.fit(sample_protein.protein.get_attribute('umap'))
            Sum_of_squared_distances.append(km.inertia_)
    
        plt.plot(K, Sum_of_squared_distances, 'bx-')
        plt.xlabel('k')
        plt.ylabel('Sum of Squared Distances')
        plt.title('Elbow Method For Optimal k')
        kn = KneeLocator(K, Sum_of_squared_distances, curve='convex', direction='decreasing')
        plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
        plt.show()
        
        print("Suggested Number of K-Means Clusters for "+ str(pca_comps_protein.value)+" PCA Components: "+str(kn.knee))
        print("\n")
        
    
    button.on_click(on_button_clicked)
    
    
    print("Select The Number of K-Means Clusters: ")
    kmeans_clusters_protein=widgets.BoundedIntText(min=2,max=100,step=1,description='K-Means Clusters:',disabled=False)
    display(kmeans_clusters_protein)
    button = widgets.Button(description="Parameter Selected")
    display(button)
    print("\n")
    out = widgets.Output()
    display(out)

    
    @out.capture(clear_output=True,wait=True)
    def on_button_clicked(b):
        sample_protein.protein.cluster(attribute = 'umap', method = 'kmeans',n_clusters = kmeans_clusters_protein.value)
    
        fig6 = sample_protein.protein.scatterplot(attribute='umap',colorby='label')
        fig6.show()
    
        #button = widgets.Button(description="Save plot")
        #display(button)
        print("\n")
        #out = widgets.Output()
        #display(out)

        #@out.capture(clear_output=True,wait=True)
        #def on_button_clicked(b):
        #    fig6.write_image("protein_umap_projection.png")
        #    print("Saved image as protein_umap_projection.png")
   
        #button.on_click(on_button_clicked)
    
  
        fig7 = sample_protein.protein.heatmap(attribute='normalized_counts')
        fig7.show()
    
        #button = widgets.Button(description="Save plot")
        #display(button)
        print("\n")
        #out = widgets.Output()
        #display(out)

        #@out.capture(clear_output=True,wait=True)
        #def on_button_clicked(b):
        #    fig7.write_image("protein_heatmap.png")
        #    print("Saved image as protein_heatmap.png")
        
        #button.on_click(on_button_clicked)

    button.on_click(on_button_clicked)

    print("Select Features - Use the Command Button for Multiple Selections: ")
    features_list_protein=widgets.SelectMultiple(options=listofid,description='Select Features',disabled=False)
    display(features_list_protein)
    button = widgets.Button(description="Features Selected")
    display(button)
    print("\n")
    out = widgets.Output()
    display(out)

    @out.capture(clear_output=True,wait=True)
    def on_button_clicked(b):
        print(list(features_list_protein.value))
        if list(features_list_protein.value): 
            fig9 = sample_protein.protein.scatterplot(attribute='umap',colorby='normalized_counts', features=features_list_protein.value)
            fig9.show()
        
           # button = widgets.Button(description="Save plot")
           # display(button)
            print("\n")
           # out = widgets.Output()
           # display(out)

           # @out.capture(clear_output=True,wait=True)
           # def on_button_clicked(b):
           #     fig9.write_image("protein_feature_umap_projection.png")
            #    print("Saved image as protein_feature_umap_projection.png")
            
            #button.on_click(on_button_clicked)
    
        
            fig10 = sample_protein.protein.heatmap(attribute='normalized_counts', features=features_list_protein.value)
            fig10.show()
        
            #button = widgets.Button(description="Save plot")
            #display(button)
            print("\n")
            #out = widgets.Output()
            #display(out)

            #@out.capture(clear_output=True,wait=True)
            #def on_button_clicked(b):
            #    fig10.write_image("protein_feature_heatmap.png")
            #    print("Saved image as protein_feature_heatmap.png")
            
            #button.on_click(on_button_clicked)
    
       
            fig11 = sample_protein.protein.ridgeplot(attribute='normalized_counts',splitby='label',features=features_list_protein.value)
            fig11.show()
        
            #button = widgets.Button(description="Save plot")
            #display(button)
            print("\n")
            #out = widgets.Output()
            #display(out)

            #@out.capture(clear_output=True,wait=True)
            #def on_button_clicked(b):
            #    fig11.write_image("protein_feature_ridgeplot.png")
            #    print("Saved image as protein_feature_ridgeplot.png")
            
            #button.on_click(on_button_clicked)
    

            final_list=list(itertools.combinations(features_list_protein.value, 2))
        
            for element in final_list:
                df = pd.DataFrame(data=sample_protein.protein.get_attribute('id'))
                listofid=list(df[0])
                index = listofid.index(element[0])
            
                df2 = pd.DataFrame(data=sample_protein.protein.get_attribute('normalized_counts'))
                value1=list(df2.loc[:,index])
            
                df = pd.DataFrame(data=sample_protein.protein.get_attribute('id'))
                listofid=list(df[0])
                index2 = listofid.index(element[1])

                df2 = pd.DataFrame(data=sample_protein.protein.get_attribute('normalized_counts'))
                value2=list(df2.loc[:,index2])
            
                fig12 = go.Figure()
                fig12.add_trace(go.Scatter(x=value1, y=value2,mode='markers',marker=dict(size=3)))
                fig12.update_layout(xaxis_title=element[0],yaxis_title=element[1])
                fig12.show()
            
    button.on_click(on_button_clicked)    

In [ ]:
%%capture
# get data but don't show user any output
pkg = q3.Package.browse(handle, registry="s3://"+bucket, top_hash=top_hash)

In [ ]:
bytes=pkg["PBMC_and_CART_Sample2.dna+protein.h5"].get_bytes()
print('\033[1m',"This tool is specifically designed to analyze MissionBio generated DNA and protein expression data. The input into the tool is a h5 file containing the DNA/CNV and protein count matrices. This tool gives the user the ability to perform normalization/scaling, dimension reduction (i.e. PCA and UMAP), and clustering (k-means) in real time. The user has the ability to modify parameters around dimension reduction and clustering. To guide the user, automated and suggested parameters are provided using a Variance Plot (for PCA) and an Elbow Plot (for k-means clustering). The user can also select a subset of the data for closer analysis. "+'\033[0m')
print('\n')
cnv_analysis(io.BytesIO(bytes))
protein_analysis(io.BytesIO(bytes))